In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=afaa3df3342625af83c754f38851f5ff140949470c0e32df47f6b2462b720759
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
import os
import sys
from google.colab import drive

In [3]:
import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql.session import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F
import re
from gensim.models import Word2Vec
from gensim import utils
from time import time
import random

In [4]:
os.cpu_count()

8

In [5]:
drive.mount("/content/drive")
%cd /content/drive/MyDrive/CSE6250-assignments/final_project_10222023

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CSE6250-assignments/final_project_10222023


In [6]:
import textCNN
import utils
import basic_models
import work_on_textCNN

In [7]:
ls

 annotations.csv                 NOTEEVENTS-2.csv.gz           textCNN.py
 basic_models.py                 preprocess.py                 train_val_test_data.pkl
'CSE6250 Final Peoject1.ipynb'   __pycache__/                  utils.py
 df_annot_text_split.pkl         textCNN_cohort_bestaccu.pt    w2v.txt
 dict_vocab.txt                  textCNN_Obesity_bestaccu.pt   work_on_textCNN.py


In [8]:
conf = SparkConf().setAppName('Preprocess').setMaster('local[*]')
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.master('local[*]').appName('merge_data').getOrCreate()

In [9]:
noteevents_struct = StructType([StructField("row_id", IntegerType(), True),
                      StructField("subject_id", IntegerType(), True),
                      StructField("Hospital_Admission_ID", IntegerType(), True),
                      StructField("chartdate", TimestampType(), True),
                      # StructField("charttime", TimestampType(), True),
                      # StructField("storetime", TimestampType(), True),
                      StructField("category", StringType(), True),
                      StructField("description", StringType(), True),
                      StructField("cg_id", IntegerType(), True),
                      StructField("iserror", IntegerType(), True),
                      StructField("text", StringType(), True)])
df_noteevents1 = spark.read.csv('NOTEEVENTS-2.csv.gz', header = True, schema = noteevents_struct)
df_noteevents2 = df_noteevents1.filter(df_noteevents1.category == "Discharge summary")
df_noteevents2.count()

59652

In [10]:
#Use the code to check if any column contains null or real values, so that the columns with no real values can be dropped to save space.
df_noteevents2.select('*', F.col('chartdate').isNotNull().alias('Row_selection')).where(F.col('Row_selection')).show()

+------+----------+---------------------+-------------------+-----------------+-----------+-----+-------+--------------------+-------------+
|row_id|subject_id|Hospital_Admission_ID|          chartdate|         category|description|cg_id|iserror|                text|Row_selection|
+------+----------+---------------------+-------------------+-----------------+-----------+-----+-------+--------------------+-------------+
|   776|     20007|               188442|2183-10-29 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|         true|
|   777|     20007|               193793|2184-01-20 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|         true|
|   778|     59883|               118446|2103-04-18 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|         true|
|   779|     17043|               157985|2103-10-11 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|         true|
|   785|     

In [11]:
#Use the code to check if any column contains abnormal values, e.g., does subject id column has non-integer values?
dn1 = df_noteevents2.select("*", F.col("subject_id").cast("int").isNotNull().alias("Row_Selection")).where(F.col("Row_Selection"))
dn1.count() == df_noteevents2.count()

True

In [12]:
df_annot1 = spark.read.csv('./annotations.csv', header = True)
df_annot2 = df_annot1.toDF(*(column.replace('.', '_') for column in df_annot1.columns))
df_annot3 = df_annot2.select(*(F.col(c).cast("integer").alias(c) for c in df_annot2.columns)).distinct()
# Use the code below to add an index column.
df_annot4 = df_annot3.select('*').withColumn("idx", F.monotonically_increasing_id())
df_annot4.createOrReplaceTempView('annotations')

In [13]:
#Check if there are rows that have the duplicate subject ids and hospital admission ids (identical ids may have different annotations).
df1 = df_annot4.dropDuplicates(subset = ["subject_id", "Hospital_Admission_ID"])
df2 = df_annot4.join(df1, how = 'left_anti', on = ["idx"])
df2.show() #There are ten combinations of ids that have different annotations. Note the chart time is identical to hospital ID or 999999. I am not sure how to associate the time with the chart time column in df_noteevents2.
#I decide to remove all the rows with duplicated ids but different in annotations.

+----+---------------------+----------+----------+------+-------+-------------+-------------------------------+----------------------+---------------------+---------------------------------------------+-------------+---------------------+-------------------------+--------------------------------+---------------+----------+--------+------+
| idx|Hospital_Admission_ID|subject_id|chart_time|cohort|Obesity|Non_Adherence|Developmental_Delay_Retardation|Advanced_Heart_Disease|Advanced_Lung_Disease|Schizophrenia_and_other_Psychiatric_Disorders|Alcohol_Abuse|Other_Substance_Abuse|Chronic_Pain_Fibromyalgia|Chronic_Neurological_Dystrophies|Advanced_Cancer|Depression|Dementia|Unsure|
+----+---------------------+----------+----------+------+-------+-------------+-------------------------------+----------------------+---------------------+---------------------------------------------+-------------+---------------------+-------------------------+--------------------------------+---------------+-----

In [14]:
#If there are also duplicate subj and hospital ids in df_noteevents2, those rows also need to be removed.
df_noteevents2.filter((df_noteevents2.Hospital_Admission_ID == 164133)&(df_noteevents2.subject_id == 25553)).show()

+------+----------+---------------------+-------------------+-----------------+-----------+-----+-------+--------------------+
|row_id|subject_id|Hospital_Admission_ID|          chartdate|         category|description|cg_id|iserror|                text|
+------+----------+---------------------+-------------------+-----------------+-----------+-----+-------+--------------------+
|  3889|     25553|               164133|2146-06-03 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|
|  3890|     25553|               164133|2146-06-08 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|
|  3891|     25553|               164133|2146-06-22 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|
| 55520|     25553|               164133|2146-06-08 00:00:00|Discharge summary|   Addendum| NULL|   NULL|Name:  [**Known l...|
+------+----------+---------------------+-------------------+-----------------+-----------+-----+-------+------

In [15]:
#Use the code to remove rows in df_annot that have duplicate values in column hospital admission id and subject id.
da1 = df_annot3.join(
    df_annot3.groupBy('Hospital_Admission_ID', 'subject_id').agg((F.count("*")>1).cast("int").alias('Row_selection')),
    on = ['Hospital_Admission_ID', 'subject_id'],
    how = 'inner'
)
da2 = da1.where(F.col('Row_selection') == 0)
da2.createOrReplaceTempView('annotations')
da2.show()

+---------------------+----------+----------+------+-------+-------------+-------------------------------+----------------------+---------------------+---------------------------------------------+-------------+---------------------+-------------------------+--------------------------------+---------------+----------+--------+------+-------------+
|Hospital_Admission_ID|subject_id|chart_time|cohort|Obesity|Non_Adherence|Developmental_Delay_Retardation|Advanced_Heart_Disease|Advanced_Lung_Disease|Schizophrenia_and_other_Psychiatric_Disorders|Alcohol_Abuse|Other_Substance_Abuse|Chronic_Pain_Fibromyalgia|Chronic_Neurological_Dystrophies|Advanced_Cancer|Depression|Dementia|Unsure|Row_selection|
+---------------------+----------+----------+------+-------+-------------+-------------------------------+----------------------+---------------------+---------------------------------------------+-------------+---------------------+-------------------------+--------------------------------+--------

In [16]:
dn1 = df_noteevents2.join(
    df_noteevents2.groupBy('Hospital_Admission_ID', 'subject_id').agg((F.count("*")>1).cast("int").alias('Row_selection')),
    on = ['Hospital_Admission_ID', 'subject_id'],
    how = 'inner'
)
dn2 = dn1.where(F.col('Row_selection') == 0)
dn2.createOrReplaceTempView('annotations')
dn2.show()

+---------------------+----------+------+-------------------+-----------------+-----------+-----+-------+--------------------+-------------+
|Hospital_Admission_ID|subject_id|row_id|          chartdate|         category|description|cg_id|iserror|                text|Row_selection|
+---------------------+----------+------+-------------------+-----------------+-----------+-----+-------+--------------------+-------------+
|               115686|     30267|   795|2158-09-15 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|            0|
|               170041|     32689|   849|2129-08-09 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|            0|
|               107500|     10567|   601|2198-07-06 00:00:00|Discharge summary|     Report| NULL|   NULL|"Admission Date: ...|            0|
|               146660|     20856|  1248|2197-03-17 00:00:00|Discharge summary|     Report| NULL|   NULL|Admission Date:  ...|            0|
|            

In [17]:
da3 = da2.drop('Row_selection', 'chart_time')
da3.createOrReplaceTempView('annotations')

In [18]:
dn3 = dn2.select('Hospital_Admission_ID', 'subject_id', 'text')
dn3.createOrReplaceTempView('noteevents')

In [19]:
#Use sql to select rows with distinct ids in each df, df_annot and df_noteevent, and the ids should be identical across the two dfs.
df_annot_text = spark.sql('''select annotations.*, text
                             from noteevents join annotations on
                             annotations.subject_id == noteevents.subject_id and
                             annotations.Hospital_Admission_ID == noteevents.Hospital_Admission_ID''')
#df_annot_text2 = df_annot_text.drop(*['chart_time', 'Hospital_Admission_ID'])
df_annot_text.createOrReplaceTempView("annot_text")
# df_annot_text.count()

In [20]:
#Confirm that the combinations of hospital and subject ids are all distinct.
df_annot_text.select('Hospital_Admission_ID', 'subject_id').distinct().count() == df_annot_text.count()

True

In [21]:
#Use regex to clean the text column and return the list of words.
#A list of strings must be returned here to fit the requirement of
#list of list of strings for word2vec training.
def pre_cleaning(text):
    pattern = re.compile('<[^>]*>|\W(\w\.)+\w?')
    text = re.sub(pattern, ' ', text.lower())
    text = re.sub(r"[^A-Za-z0-9()!?,'`\"]", ' ', text)
    text = re.sub(r"\'s", " \'s", text)
    text = re.sub(r"\'ve", " \'ve", text)
    text = re.sub(r"n\'t", " n\'t", text)
    text = re.sub(r"\'re", " \'re", text)
    text = re.sub(r"\'d", " \'d", text)
    text = re.sub(r"\'ll", " \'ll", text)
    text = re.sub(r",", " , ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\(", " ( ", text)
    text = re.sub(r"\)", " ) ", text)
    text = re.sub(r"\?", " ? ", text)
    return [i for i in text.split()]

In [22]:
#User defined text pre-cleaning function. Note that each cell should be a list of strings.
#The datatype thus should be ArrayType(StringType()). If only StringType() is provided, the outupt
#will be a string converted from a list, e.g., '[...]', and cannot be trained properly by word2vec next.
textUDF = F.udf(lambda x: pre_cleaning(x), ArrayType(StringType()))
#Convert the whole text column.
df_annot_text2 = df_annot_text.withColumn('text', textUDF(F.col('text')))
#Convert the text column to a list. Note that rdd.map().collect() will just collect the lists in each cell
#into a list. There is no need to add list() for the code below.
token_list = df_annot_text2.rdd.map(lambda x: x.text).collect()

In [23]:
#Save the annot_text file as excel spreadsheet. Because of the list of strings in the text column,
#The file cannot be saved as csv.
df_annot_text3 = df_annot_text2.toPandas()
df_annot_text3.to_pickle('df_annot_text_split.pkl')

In [28]:
vector_size = 32
model_w2v = Word2Vec(min_count = 1, window = 5, vector_size = vector_size, sample = 1e-5, negative = 5, workers = 8, sg = 0)
model_w2v.build_vocab(token_list)
indices = list(range(len(token_list)))
t0 = time()

for i in range(20):
  random.shuffle(indices)
  texts_samples = [token_list[j] for j in indices]
  model_w2v.train(texts_samples, total_examples = model_w2v.corpus_count, epochs = 5)

print('time to train the model:{0:.2f} mins.'.format((time() - t0)/60))

time to train the model:1.06 mins.


In [29]:
model_w2v.wv.most_similar('alcohol')

[('marijuana', 0.8670544624328613),
 ('abuse', 0.8458220362663269),
 ('tobacco', 0.8412259221076965),
 ('etoh', 0.815120279788971),
 ('ectasy', 0.812666654586792),
 ('recreational', 0.8057549595832825),
 ('use', 0.8034866452217102),
 ('smoking', 0.8031714558601379),
 ('illicit', 0.8023684024810791),
 ('drinks', 0.7911333441734314)]

In [30]:
model_w2v.wv.save_word2vec_format('./w2v.txt', binary = False)

In [ ]:
!python -c "import basic_models; basic_models.main()" --input_file_path 'tran_val_test_data.pkl' --ngram 1 --model_name 'logistic'

Reading in tran_val_test_data.pkl
Now working on condition cohort
report               precision    recall  f1-score   support

           0      0.627     0.627     0.627        75
           1      0.525     0.525     0.525        59

    accuracy                          0.582       134
   macro avg      0.576     0.576     0.576       134
weighted avg      0.582     0.582     0.582       134

accuracy score 0.5821
auc_score 0.5760
Figure(640x480)
Now working on condition Obesity
report               precision    recall  f1-score   support

           0      0.908     1.000     0.952       119
           1      1.000     0.200     0.333        15

    accuracy                          0.910       134
   macro avg      0.954     0.600     0.643       134
weighted avg      0.919     0.910     0.883       134

accuracy score 0.9104
auc_score 0.6000
Figure(640x480)
Now working on condition Non_Adherence
report               precision    recall  f1-score   support

           0      0.91

In [ ]:
!python -c "import basic_models; basic_models.main()" --input_file_path 'tran_val_test_data.pkl' --ngram 2 --model_name 'logistic'

Reading in tran_val_test_data.pkl
Now working on condition cohort
report               precision    recall  f1-score   support

           0      0.637     0.680     0.658        75
           1      0.556     0.508     0.531        59

    accuracy                          0.604       134
   macro avg      0.597     0.594     0.595       134
weighted avg      0.601     0.604     0.602       134

accuracy score 0.6045
auc_score 0.5942
Figure(640x480)
Now working on condition Obesity
report               precision    recall  f1-score   support

           0      0.895     1.000     0.944       119
           1      1.000     0.067     0.125        15

    accuracy                          0.896       134
   macro avg      0.947     0.533     0.535       134
weighted avg      0.907     0.896     0.853       134

accuracy score 0.8955
auc_score 0.5333
Figure(640x480)
Now working on condition Non_Adherence
report               precision    recall  f1-score   support

           0      0.91

In [ ]:
!python -c "import basic_models; basic_models.main()" --input_file_path 'tran_val_test_data.pkl' --ngram 1 --model_name 'naive bayes'

Reading in tran_val_test_data.pkl
Now working on condition cohort
report               precision    recall  f1-score   support

           0      0.684     0.720     0.701        75
           1      0.618     0.576     0.596        59

    accuracy                          0.657       134
   macro avg      0.651     0.648     0.649       134
weighted avg      0.655     0.657     0.655       134

accuracy score 0.6567
auc_score 0.6481
Figure(640x480)
Now working on condition Obesity
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.

In [ ]:
!python -c "import basic_models; basic_models.main()" --input_file_path 'tran_val_test_data.pkl' --ngram 1 --model_name 'linear svc'

Reading in tran_val_test_data.pkl
Now working on condition cohort
report               precision    recall  f1-score   support

           0      0.603     0.587     0.595        75
           1      0.492     0.508     0.500        59

    accuracy                          0.552       134
   macro avg      0.547     0.548     0.547       134
weighted avg      0.554     0.552     0.553       134

accuracy score 0.5522
auc_score 0.5476
Figure(640x480)
Now working on condition Obesity
report               precision    recall  f1-score   support

           0      0.915     1.000     0.956       119
           1      1.000     0.267     0.421        15

    accuracy                          0.918       134
   macro avg      0.958     0.633     0.688       134
weighted avg      0.925     0.918     0.896       134

accuracy score 0.9179
auc_score 0.6333
Figure(640x480)
Now working on condition Non_Adherence
report               precision    recall  f1-score   support

           0      0.91

In [31]:
!python -c "import work_on_textCNN; work_on_textCNN.main()" --annot_text_path 'df_annot_text_split.pkl' --w2v_path 'w2v.txt' --data_filename 'train_val_test_data.pkl' --batchsize 16 --train_size 0.7 --valid_size 0.2 --cnn_channels 3 --cnn_windows 1 2 3 --target_name 'cohort' --epochs 15

Reading in df_annot_text_split.pkl
vocabulary size is 29060
num of all data points is 1341
max sentence length is 5274
write text2index to dict_vocab.dict
There are 1 words that are not provided in the w2v table.
Train worker process using cpu for training
Now working on condition cohort.
Now Training Epoch 0 ====
Batch 1 in the current training epoch, loss per batch 2.09, tokens per second 82535.41, accuracy 46.88%.
Batch 11 in the current training epoch, loss per batch 11.11, tokens per second 100758.60, accuracy 47.92%.
Batch 21 in the current training epoch, loss per batch 10.36, tokens per second 131189.28, accuracy 48.30%.
Batch 31 in the current training epoch, loss per batch 9.99, tokens per second 123775.25, accuracy 48.44%.
Batch 41 in the current training epoch, loss per batch 11.93, tokens per second 110679.84, accuracy 46.58%.
Batch 51 in the current training epoch, loss per batch 10.49, tokens per second 126267.09, accuracy 47.00%.
loss 6.42e+01, token 1.33e+06 in epoch 0

In [ ]:
import torch
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)


In [ ]:
print('{:.5e}'.format(0.00333))

3.33000e-03


In [ ]:
import torch

# import numpy module
import numpy

# create one dimensional tensor with
# float type elements
b = torch.tensor([10.12, 20.56, 30.00, 40.3, 50.4])

print(b)
a = []

# convert this into numpy array using
# numpy() method
b = b.tolist()

# display
a += b
a

tensor([10.1200, 20.5600, 30.0000, 40.3000, 50.4000])


[10.119999885559082,
 20.559999465942383,
 30.0,
 40.29999923706055,
 50.400001525878906]